In [73]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
#import seaborn as sns

In [399]:
#a. Read data
crash_data = pd.read_csv("crash_data.csv", usecols=(0,1,2,3,4,5))    #Reading data
crash_data.head()

,country,date,fatalities,flight_model,location,operator
0,Italy,02-08-1919,14,Caproni Ca.48,Verona,Caproni
1,U.K.,11-08-1919,1,Felixstowe Fury,near Felixtowe RNAS,RAF
2,South Africa,23-02-1920,0,Handley Page O/7,Acadia Sidin...,Handley Page Transport
3,Sudan,25-02-1920,0,Handley Page O/400,near El Shereik,Handley Page Transport
4,Sweden,30-06-1920,0,Handley Page O/400,CstanC%,Handley Page Transport


In [400]:
crash_data.describe()

,country,date,fatalities,flight_model,location,operator
count,22177,22176,18014,22175,21213,22117
unique,234,14056,364,3002,13266,5466
top,USA,10-05-1940,0,Douglas C-47A (DC-3),unknown,USAAF
freq,3968,171,9307,1904,265,2575


In [401]:
crash_data.isnull().sum()

country            0
date               1
fatalities      4163
flight_model       2
location         964
operator          60
dtype: int64

In [502]:
cleandf = crash_data.copy()

In [503]:
#b. aggregating values 
# replaceing null values with 0. As it is related to the death data is missing may be there was no data captured because of no death/
print(cleandf["fatalities"].isnull().sum())
cleandf["fatalities"].fillna(0, inplace = True) 

#print(cleandf["fatalities"].value_counts())
#print(cleandf["fatalities"].head(100))


# as it is not clearly mentioned in the question about the kind of aggregation so i'll be using mean()
s = pd.Series(cleandf["fatalities"])
print(s.str.count('\+').sum())

import statistics

cleandf['fatalities'] = cleandf['fatalities'].replace('\+', '-',regex=True)

def get_year(dt):
    vaal = []
    for i in dt:
        #rint(type(str(i)))
        try:
            first = re.sub(r'(\d{1,2})- (\d{1,2})', '\\1', str(i))
            #print(first)
            second = re.sub(r'(\d{1,2})- (\d{1,2})', '\\2', str(i))
            vaal.append(statistics.mean([int(first), int(second)]))
        except:
            vaal.append(int(i))
    return vaal
vale = get_year(cleandf["fatalities"])
cleandf["fatalities"] = pd.DataFrame(vale)
cleandf.head()

4163
335.0


,country,date,fatalities,flight_model,location,operator
0,Italy,02-08-1919,14.0,Caproni Ca.48,Verona,Caproni
1,U.K.,11-08-1919,1.0,Felixstowe Fury,near Felixtowe RNAS,RAF
2,South Africa,23-02-1920,0.0,Handley Page O/7,Acadia Sidin...,Handley Page Transport
3,Sudan,25-02-1920,0.0,Handley Page O/400,near El Shereik,Handley Page Transport
4,Sweden,30-06-1920,0.0,Handley Page O/400,CstanC%,Handley Page Transport


In [504]:
print(cleandf.isnull().sum())


country           0
date              1
fatalities        0
flight_model      2
location        964
operator         60
dtype: int64


In [505]:
#delete missing date record from Date country, and Flight model colum

#Date
cleandf = cleandf.drop(cleandf[cleandf["date"].isnull()].index.tolist())

#Flight model
cleandf = cleandf.drop(cleandf[cleandf["flight_model"].isnull()].index.tolist())

#country
cleandf['country'] = cleandf['country'].replace('\?', 'NaN',regex=True)
cleandf = cleandf.drop(cleandf[cleandf["country"].isnull()].index.tolist())

#Operator

cleandf['operator'] = cleandf['operator'].replace(np.nan, 'Unknown', regex=True)

print(cleandf.isnull().sum())

country           0
date              0
fatalities        0
flight_model      0
location        962
operator          0
dtype: int64
        country        date  fatalities        flight_model  \
0         Italy  02-08-1919        14.0       Caproni Ca.48   
1          U.K.  11-08-1919         1.0     Felixstowe Fury   
2  South Africa  23-02-1920         0.0    Handley Page O/7   
3         Sudan  25-02-1920         0.0  Handley Page O/400   
4        Sweden  30-06-1920         0.0  Handley Page O/400   

              location                operator  
0               Verona                 Caproni  
1  near Felixtowe RNAS                     RAF  
2      Acadia Sidin...  Handley Page Transport  
3      near El Shereik  Handley Page Transport  
4             CstanC%  Handley Page Transport  


In [506]:
# handlling missing data form operator using country and other infromation
# sorting dataframe 
#test1 = cleandf.copy()
#print(test1.head())


#For country poland
#poland = test1[(test1['country'] == "Poland") & (test1['operator'] = == "Poland"]
#print(poland.index.tolist())

#print(test1[test1['operator'] == "Unknown"])

#operator_null_loc = cleandf[cleandf['operator'].isnull()].index.tolist()

#for i in operator_null_loc:
#    cleandf.loc[i,:]['operator'] = "Unknown"
    
#print(cleandf.isnull().sum())

In [507]:
#c. Create 2 columns month , year
#Cleaning date column

print( cleandf['date'].str.strip('\??').astype(bool).any().sum() )

# changing the date to the nearby date
cleandf['date'] = cleandf['date'].replace('\?', '1',regex=True)
cleandf['date'] = cleandf['date'].replace('111', '12',regex=True)

#print(cleandf["date"].head())

#Get month and year
import re
def get_year(dt):
        yy = []
        for i in dt:
            try:
                yy.append(re.sub(r'(\d{1,2})-(\d{1,2})-(\d{4})', '\\3', str(i)))
            except Exception as e:
                print(e)
        return yy
def get_month(dt):
        mm = []
        for i in dt:
            try:
                mm.append(re.sub(r'(\d{1,2})-(\d{1,2})-(\d{4})', '\\2', str(i)))
            except Exception as e:
                print(e)
        return mm
list_year= get_year(cleandf['date'])
list_month= get_month(cleandf['date'])
cleandf['month'] = pd.DataFrame(list_month)
cleandf['year'] = pd.DataFrame(list_year)
cleandf.head()

1


,country,date,fatalities,flight_model,location,operator,month,year
0,Italy,02-08-1919,14.0,Caproni Ca.48,Verona,Caproni,08,1919
1,U.K.,11-08-1919,1.0,Felixstowe Fury,near Felixtowe RNAS,RAF,08,1919
2,South Africa,23-02-1920,0.0,Handley Page O/7,Acadia Sidin...,Handley Page Transport,02,1920
3,Sudan,25-02-1920,0.0,Handley Page O/400,near El Shereik,Handley Page Transport,02,1920
4,Sweden,30-06-1920,0.0,Handley Page O/400,CstanC%,Handley Page Transport,06,1920


In [508]:
#d. new dataframe with country,fatalities,year,month
cols = ['country','fatalities','year','month']
countires_fatalities = cleandf[cols]
countires_fatalities.head()

,country,fatalities,year,month
0,Italy,14.0,1919,08
1,U.K.,1.0,1919,08
2,South Africa,0.0,1920,02
3,Sudan,0.0,1920,02
4,Sweden,0.0,1920,06
